In [1]:
import os
import numpy as np
import torch
import cv2
from skimage.metrics import structural_similarity as compare_ssim
from utils import load_model_ISTA, load_model_DRS, load_model_CNN
from utils import find_test_image_circular_pad as denoise
import bm3d

In [2]:
def Ax(zin, mask):
    maskshape = mask.shape
    addarr = np.zeros(maskshape)
    arrsize0 = int((maskshape[0]+1)/2)
    arrsize1 = int((maskshape[1]+1)/2)
    addarr[arrsize0, arrsize1] = 1.
    filtfft = np.fft.fft2(mask, zin.shape) * np.fft.fft2(addarr, zin.shape)
    zf = np.fft.fft2(zin)
    return np.real(np.fft.ifft2(np.multiply(zf, np.conjugate(filtfft))))

def Atopx(zin, mask):
    maskshape = mask.shape
    addarr = np.zeros(maskshape)
    arrsize0 = int((maskshape[0]+1)/2)
    arrsize1 = int((maskshape[1]+1)/2)
    addarr[arrsize0, arrsize1] = 1.
    filtfft = np.fft.fft2(mask, zin.shape) * np.fft.fft2(addarr, zin.shape)
    zf = np.fft.fft2(zin)
    return np.real(np.fft.ifft2(np.multiply(zf, filtfft)))

In [3]:
def proj(im_input, minval, maxval):
    im_out = np.where(im_input > maxval, maxval, im_input)
    im_out = np.where(im_out < minval, minval, im_out)
    return im_out

def psnr(x,y):
    norm2 = np.mean((x - y) ** 2)
    psnrval = -10 * np.log10(norm2)
    return psnrval

def proxrhof(y, filtfft, rho, xhat):
    vf = np.fft.fft2(y + rho*xhat)
    x = np.real(np.fft.ifft2(np.divide(vf, filtfft + rho)))
    return x

In [4]:
def pnp_redpg_deblur(model, im_input, im_ref, mask, denoiser, **opts):

    sigma = opts.get('sigma', 15)
    lamda = opts.get('lamda', 1.0)
    L = opts.get('L', 1.0)
    maxitr = opts.get('maxitr', 100)
    verbose = opts.get('verbose',1)
    rho = lamda * L

    """ Initialization. """

    m, n = im_input.shape
    index = np.nonzero(mask)
    y = Atopx(im_input, mask)
    filtfft = np.square(np.absolute(np.fft.fft2(mask, y.shape)))

    x = 0.5 * np.ones((m,n), dtype=np.float64)
    v = np.copy(x)

    """ Main loop. """
    for i in range(maxitr):

        xold = np.copy(x)
        vold = np.copy(v)
        """ Update variables. """

        x = proxrhof(y, filtfft, rho, vold)
        """ Denoising step. """

        vtilde = np.copy(x)
        vtilde_torch = np.reshape(vtilde, (1,1,m,n))
        vtilde_torch = torch.from_numpy(vtilde_torch).type(torch.FloatTensor).cuda()
        if denoiser == "Proposed_ISTA" or denoiser == "Proposed_DRS":
            v = denoise(vtilde_torch, model, 2, 64, 1024).cpu().detach().numpy()
            v = np.reshape(v, (m,n))
        if denoiser == "DnCNN" or denoiser == "RealSN_DnCNN" or denoiser == "SimpleCNN" or denoiser == "RealSN_SimpleCNN":
            res = model(vtilde_torch).cpu().detach().numpy()
            res = np.reshape(res, (m,n))
            v = vtilde - res
        if denoiser == "BM3D":
            v = bm3d.bm3d(vtilde, sigma_psd=sigma/255, stage_arg=bm3d.BM3DStages.ALL_STAGES)

        v = (1/L) * v + (1 - (1/L))*vtilde

        """ Monitoring. """
        if verbose:
            print("Iteration : {}, \t psnr: {} \t"\
                  .format(i+1, psnr(v,im_ref)))

    return v

In [6]:
dir_name = 'data/Deblurdataset/'
image_arr = ['01.png']
outdir_name = 'Deblur_single_image/'
for imagename in image_arr:
    input_str = dir_name + imagename
    print(input_str)
    # ---- load the ground truth ----
    im_orig = cv2.imread(input_str, 0)/255.0

    m,n = im_orig.shape

    # ---- blur the image

    ## Gaussian blur
    kernel = cv2.getGaussianKernel(25, 1.6)
    mask = np.outer(kernel, kernel.transpose())

    ## External filters
    #f = io.loadmat('kernels/kernel1.mat')
    #mask = f['kernel']
    #print(mask)

    ## Motion blur
    kernel_size = 25

    # Create the vertical kernel.
    kernel_v = np.zeros((kernel_size, kernel_size))

    # Create a copy of the same for creating the horizontal kernel.
    kernel_h = np.copy(kernel_v)

    # Fill the middle row with ones.
    kernel_v[:, int((kernel_size - 1)/2)] = np.ones(kernel_size)
    kernel_h[int((kernel_size - 1)/2), :] = np.ones(kernel_size)

    # Normalize.
    kernel_v /= kernel_size
    kernel_h /= kernel_size
    mask = kernel_h

    im_blur = Ax(im_orig, mask)
    # ---- add noise -----
    noise_level = 10.0 /255.0
    gauss = np.random.normal(0.0, noise_level, im_blur.shape)
    im_noisy = im_blur #+ gauss
    im_noisy2 = proj(im_noisy, 0, 1)

    psnr_ours = psnr(im_noisy2, im_orig)
    ssim_ours = compare_ssim(im_noisy2, im_orig, data_range=1.)
    print('noisy - PNSR: {}, SSIM = {}'.format(psnr_ours, ssim_ours))

    maxitr = 20
    # ---- set options ----
    sigma = 5
    lamda = 0.001
    L = 1.
    rval = 20
    path = "models/proposed_denoiser_ISTA_sigma" + str(sigma) +".pth"
    opts = dict(sigma=sigma, lamda=lamda, L=L, maxitr=maxitr, verbose=True)
    denoiser = "Proposed_ISTA"
    model = load_model_ISTA(sigma, path)
    print(denoiser)

    # ---- plug and play -----
    out = pnp_redpg_deblur(model, im_noisy, im_orig, mask, denoiser, **opts)

    # ---- results ----
    psnr_ours = psnr(out, im_orig)
    ssim_ours = compare_ssim(out, im_orig, data_range=1.)
    print('Image {} - PNSR: {}, SSIM = {}'.format(imagename, psnr_ours, ssim_ours))

